In [ ]:
import pandas as pd
import operator
import re, string
import numpy as np

# Prepare Philippine Standard Geographic Code Reference File

In [ ]:
psgc = pd.read_csv("psgc/data/processed/clean-psgc.csv.gz",
                   dtype={'code':'object'},
                   compression="gzip",
                   encoding="utf-8")

In [ ]:
psgc.info()

In [ ]:
psgc.head()

In [ ]:
# totally drop places that are just "capital" or "not a province". I think in the previous cleanups of the PSGC file anything
# in parentheses was turned into a new row. As such, "Capital" was often turned into a new row.

psgc = psgc[psgc.location.isin(["CAPITAL","NOT A PROVINCE"]) == False].reset_index(drop=True)

In [ ]:
# interlevel lower
psgc.interlevel = psgc.interlevel.str.lower()
psgc.interlevel = psgc.interlevel.replace({"mun": "municity", "city": "municity"}) #combine municity for now

# Clean location column a bit. but not too much because we'll use this as the "canonical" name
psgc['location'] = psgc.location.str.replace(r"NOT A PROVINCE|CAPITAL|\(|\)", "").str.strip()
psgc = psgc.drop_duplicates(subset=["code", "location", "interlevel"], keep="first")
psgc = psgc.dropna()

In [ ]:
psgc.interlevel.value_counts()

In [ ]:
# rename districts as simply "Metro Manila", "Metropolitan Manila", "National Capital Region" or "NCR"

ncr = psgc[(psgc.code == "130000000")]
ncr.loc[:,"location"] = ncr.location.str.replace("NATIONAL  REGION", "NATIONAL CAPITAL REGION")
ncr.loc[:,"location"] = ncr.location.str.replace("NATIONAL  REGION", "NATIONAL CAPITAL REGION")
ncr = ncr.append(pd.Series({"code":"130000000", "location": "METRO MANILA","interlevel":"reg","original":False}), ignore_index=True)
ncr = ncr.append(pd.Series({"code":"130000000", "location": "METROPOLITAN MANILA", "interlevel": "reg", "original": False}), ignore_index=True)
ncr.head()

In [ ]:
# remove districts and replace NCR region rows from reference file for now with cleaned up NCR rows

psgc = psgc[psgc.interlevel != 'dist'].reset_index(drop=True) # exclude districts
psgc = psgc[psgc.code != '130000000'].reset_index(drop=True) # exclude original ncr region rows
print(len(psgc))
psgc = psgc.append(ncr, ignore_index=True) # append cleaned up ncr region rows
print(len(psgc))
psgc.head()
psgc[psgc.code == "130000000"]

In [ ]:
# add as aliases abbreviations of north, south, east, west. for example, northern samar will have an alias n. samar.

nsew = re.compile(r"^NORTH(ERN)? |^SOUTH(ERN)? |^EAST(ERN?)? |^WEST(ERN)? ")
nsew_abbrev = psgc[psgc.location.str.contains(nsew)].location.str.split().str.get(0).str.slice(0,1)
nsew_abbrev.head()

In [ ]:
nsew_locs = psgc[psgc.location.str.contains(nsew)]
nsew_locs.head()

In [ ]:
nsew_locs.loc[:, 'location'] = nsew_abbrev.str.cat(psgc[psgc.location.str.contains(nsew)].location.str.replace("^NORTH(ERN)? |^SOUTH(ERN)? |^EAST(ERN?)? |^WEST(ERN)? ","").str.strip(),sep=" ")
nsew_locs.loc[:, 'original'] = False
nsew_locs.head()

In [ ]:
psgc = pd.concat([psgc, nsew_locs], ignore_index=True)
psgc.head()

In [ ]:
#fill interlevels for isabela, cotabato

psgc.loc[psgc.interlevel.isnull(), "interlevel"] = "municity"

What we need is reference file that contains the higher-level administrative territories in separate columns. This is so we can create a single "master string" that we will use for matching. We'll try this instead of matching each component individually.

First, create a dictionary of the rankings of various administrative levels.

In [ ]:
adm_rank= {'reg': 1, 'prov': 2, 'dist': 2, 'city': 3, 'mun': 3, 'municity': 3, 'submun': 3, 'bgy': 4}
adm_rank_list = sorted(adm_rank, key=lambda k: adm_rank[k])
psgc['adm_rank'] = psgc.interlevel.map(adm_rank)

In [ ]:
psgc.interlevel.value_counts()

We'll apply this later as a separate column.

Create a function that will add to our dataframe columns with the PSGC codes
of each location's higher level administrative territories. We'll then use this to fill the name columns 
with their corresponding place names.


In [ ]:
def fill_higher_level_codes(df):
        
    # Below is a dictionary of administrative hierarchy levels ranks and the stop string positions inside the PSG code.
    
    adm_rank = {1: 2,
                2: 4,
                3: 6,
                4: 9} 
    
    # Loop through each administrative level. 
    # Create additional columns for each administrative level with the suffixes _code and _name.
    # Fill each column with the names and codes of the higher level administrative territories in which
    # a place is located.
    
    for adm_level in adm_rank.keys():
        
        # create code cols        
        adm_code_col = "adm{}_code".format(adm_level)
        
        df[adm_code_col] = None
        
        # find the administrative levels that are higher than the current one
        
        higher_adm_levels = [l for l in adm_rank.keys() if l <= adm_level]
                
        for higher_level in higher_adm_levels:
            
            # higher adm level colum names
            
            higher_level_code_col = "adm{}_code".format(higher_level)
            
            # stop position of PSG code for this adm level
            
            stop_position = adm_rank[higher_level] 
            
            # derive higher level admin codes for each row
            
            codes = df.loc[df.adm_rank >= higher_level, "code"].str.slice(start=0, stop=stop_position).str.pad(9, side="right", fillchar="0")
            df.loc[df.adm_rank >= higher_level,higher_level_code_col] = codes
            
            # derive higher level admin names for each row
            
            higher_level_name_col = "adm{}_name".format(higher_level)

    return df

In [ ]:
psgc_unpivot = fill_higher_level_codes(psgc).dropna(how="all")
psgc_unpivot.head(10)

In [ ]:
all_loc_names = psgc_unpivot[['code', 'location', 'original']].rename(columns={'code': 'join_code'})
all_loc_names.head()

Add Region names.

In [ ]:
psgc_unpivot = psgc_unpivot.merge(all_loc_names.rename(columns={'location': 'adm1_name', 'original': 'adm1_is_orig'}),
                                  how="left", left_on="adm1_code", right_on="join_code").drop('join_code', axis=1)
psgc_unpivot.head()

Add Prov names.

In [ ]:
psgc_unpivot = psgc_unpivot.merge(all_loc_names.rename(columns={'location': 'adm2_name', 'original': 'adm2_is_orig'}),
                                  how="left", left_on="adm2_code", right_on="join_code").drop('join_code', axis=1)
psgc_unpivot.head()

Add MuniCity names.

In [ ]:
psgc_unpivot = psgc_unpivot.merge(all_loc_names.rename(columns={'location': 'adm3_name', 'original': 'adm3_is_orig'}),
                                  how="left", left_on="adm3_code", right_on="join_code").drop('join_code', axis=1)
psgc_unpivot.head()

Add Barangay names.

In [ ]:
psgc_unpivot = psgc_unpivot.merge(all_loc_names.rename(columns={'location': 'adm4_name', 'original': 'adm4_is_orig'}),
                                  how="left", left_on="adm4_code", right_on="join_code").drop('join_code', axis=1)
psgc_unpivot.head()

In [ ]:
# Special handling for isabela city! it's supposed to be in the province of basilan

psgc_unpivot.loc[psgc_unpivot.code.isin(["099700000", "099701000"]), "adm2_name"] = "BASILAN"

# Also, isabela is the only place with two PSGC codes -- one for province level and one for city level! lets just use one.

psgc_unpivot = psgc_unpivot[psgc_unpivot.code != "099700000"]

Create a "location tuple" that concatenates all the location components names into a single tuple. We'll use this for fuzzy matching later.

In [ ]:
#append all the rows again for places in metro manila except with blank regions. 
#this enables us to accept "Fort Bonifacio, Taguig" as an exact match even if it doesn't have "Metro Manila" in it

metro_manila = psgc_unpivot[psgc_unpivot.code.str.startswith("13")]
metro_manila.loc[:, "adm1_name"] = np.nan
print(len(psgc_unpivot))
psgc_unpivot = psgc_unpivot.append(metro_manila, ignore_index=True).reset_index(drop=True)
print(len(psgc_unpivot))

In [ ]:
def normalize_text(item):
    replacements = {r"city of|city": "",
                    r"barangay|brgy": "bgy",
                    r"[^a-zA-Z0-9_\s]": "",
                    r"poblacion": "pob",
                    r"ñ": "n"}
    
    item = item.lower()
    
    if item not in ['bgy', 'municity', 'prov', 'reg']:
        
        for k, v in replacements.items():
            item = re.sub(k,v,item.strip())
            
    return item.strip()

In [ ]:
def create_loc_tuple_with_code(row):
    
    # For now, disregard making location tuples for all regions 
    if row.interlevel == "reg": 
        return None 
    
    # Special handling for NCR:
    
    if row.code[:3] == "133" and row.interlevel != "municity": # Manila has submunicipalities so include all admin levels
        
        return tuple([normalize_text(v) for v in [row.adm4_name, row.adm3_name, row.adm2_name, row.adm1_name, row.interlevel, row.code] if (v is not None) and (v is not np.nan)])
    
    elif (row.code[:3] == "133" and row.interlevel == "municity") or (row.code[:3] == "137"): # Except when Manila City or anywhere else in NCR is the item, then exclude adm2_name
        
        return tuple([normalize_text(v) for v in [row.adm4_name, row.adm3_name, row.adm1_name, row.interlevel, row.code] if (v is not None) and (v is not np.nan)])
        
    # else, exclude region from final tuple   
    
    return tuple([normalize_text(v) for v in [row.adm4_name, row.adm3_name, row.adm2_name, row.interlevel, row.code] if (v is not None) and (v is not np.nan)])

In [ ]:
psgc_unpivot.loc[:, 'loc_tuple'] = psgc_unpivot.apply(create_loc_tuple_with_code,axis=1)
psgc_unpivot.head(10)

In [ ]:
psgc_unpivot = psgc_unpivot.drop_duplicates(keep="first")
len(psgc_unpivot)

## Create canonical names

In [ ]:
psgc_unpivot.code.nunique()

In [ ]:
#Find the original names for each unique PSGC code

psgc_unpivot['is_canonical'] = psgc_unpivot[['adm4_is_orig', 'adm3_is_orig', 'adm2_is_orig', 'adm1_is_orig']].sum(axis=1)

#does every code have a canonical name?

psgc_unpivot[psgc_unpivot.is_canonical > 0].code.nunique()

In [ ]:
# get the canonical names for each PSGC

canonical_names = psgc_unpivot.sort_values(["code", "is_canonical"], ascending=False).drop_duplicates("code", keep="first")

# check if each code has one canonical name

canonical_names.groupby('code').size().value_counts()

In [ ]:
canonical_names.head()

In [ ]:
# drop row for metro manila where region is blank, we don't want to use these in the canonical names

null_ncr_region = canonical_names[canonical_names.code.str.startswith("13") * canonical_names.adm1_name.isnull()].index
canonical_names = canonical_names.drop(null_ncr_region,axis=0).set_index('code')

# rename columns
canonical_names = canonical_names.rename(columns={"adm4_name": "bgy", "adm3_name": "municity", "adm2_name": "prov", "adm1_name": "reg"})

# keep only those we need
canonical_names = canonical_names[['bgy', 'municity', 'prov', 'reg']]

In [ ]:
print(canonical_names.info())
canonical_names.head()

In [ ]:
# join the canonical names to all possible search terms

psgc_locations = psgc_unpivot[['loc_tuple', 'code']].dropna().drop_duplicates(keep="first")
psgc_locations = psgc_locations.merge(canonical_names, left_on="code", right_index=True, how="left")
psgc_locations.info()

In [ ]:
def to_string(row):
    return ",".join(row.loc_tuple)

In [ ]:
psgc_locations['loc_tuple'] = psgc_locations.apply(to_string,axis=1)

In [ ]:
psgc_locations.head()

In [ ]:
psgc_locations['candidate_terms'] = psgc_locations['loc_tuple'].str.rsplit(',', n=1).str.get(0)

In [ ]:
psgc_locations = psgc_locations.set_index('loc_tuple')
psgc_locations.head()

In [ ]:
# drop region column

psgc_locations = psgc_locations.drop('reg',axis=1)

In [ ]:
psgc_locations.to_csv('psgc-locations.csv.gz',compression="gzip")